In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model

## Read Data

In [40]:
train_slice_df = pd.read_csv("slice_std_train.csv")
test_slice_df = pd.read_csv("slice_std_test.csv")
print(train_slice_df.shape, test_slice_df.shape)

(42800, 361) (10700, 361)


In [41]:
X_train = train_slice_df[train_slice_df.columns[:-1]]
y_train = train_slice_df[train_slice_df.columns[-1]]
X_test = test_slice_df[train_slice_df.columns[:-1]]
y_test = test_slice_df[train_slice_df.columns[-1]]

## Normalize data

In [75]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both training and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Create Model

In [80]:
# Define the input shape
input_layer = layers.Input(shape=(360,))

# Define the architecture of the model
x = tf.keras.layers.Dense(256, activation='relu')(input_layer)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
output_layer = layers.Dense(1, activation="linear")(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Print the model summary to check the architecture
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
plt.plot(history.history["loss"],label="loss")
plt.plot(history.history["val_loss"],label="val_loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("Model loss")
plt.show()

## Get Metric

In [79]:
pred=model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test, pred))
r2=r2_score(y_test, pred)
print("r2 score is {}:".format(r2))
print("\n")
print("RMSE is {}:".format(rmse))

335/335 [==============================] - 0s 1ms/step
r2 score is 0.9992368407243034:


RMSE is 0.6177766022517834:


## Dump Answer

In [63]:
model.save('my_model.h5')